## EMERGENCY NON ENERGENCY CLASSIFICATION USING TRANSFER LEARNING

In [2]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize


from keras.utils import to_categorical

#preprocess data accorsing to VGG16
from keras.applications.vgg16 import preprocess_input

# for instantiating model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense , InputLayer

Using TensorFlow backend.
C:\Users\avani\AnacondaLatest\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\avani\AnacondaLatest\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\avani\AnacondaLatest\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\avani\AnacondaLatest\lib\site-packages\tensorflow\python

In [3]:
# load data
datadf = pd.read_csv('emergency_classification.csv')

In [4]:
datadf.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [9]:
X = []

for img in datadf['image_names']:
    image = plt.imread('images/' + img)
    X.append(image)
    
X = np.array(X)

In [11]:
y = np.array(datadf['emergency_or_not'])

In [12]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [13]:
y = to_categorical(y)

In [14]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

## PREPROCESS DATA

In [15]:
# here we are prepricessing data as per models requirements
X.min() , X.max()

(0, 255)

In [16]:
# preprocess images as per requirements of VGG16 model
X = preprocess_input(X,mode = 'tf')

In [17]:
X.min() , X.max()

(-1.0, 1.0)

In [18]:
# split data set into train and validation set
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size = 0.2)

In [19]:
print('Train X shape -',X_train.shape)
print('Train y shape -',y_train.shape)
print('Validation X shape -',X_valid.shape)
print('Validation y shape -',y_valid.shape)

Train X shape - (1881, 224, 224, 3)
Train y shape - (1881, 2)
Validation X shape - (471, 224, 224, 3)
Validation y shape - (471, 2)


## LOAD WEIGHTS OF PRETRAINED MODEL

In [20]:
# create model with pre trained imagenet weights
base_model = VGG16(weights = 'imagenet')


553467904/553467096 [==============================] - 95s 0us/step


In [21]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [22]:
# remove the last 4  layers of the neural network. we need to adjust it as per our requirements
base_model = VGG16(weights = 'imagenet', input_shape=(224,224,3), include_top = False)

58892288/58889256 [==============================] - 10s 0us/step


In [24]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## FINE TUNE THE MODEL FOR CURRENT PROBLEM

In [26]:
# extract features using the pretrained VGG16 model

# for training set
base_model_pred = base_model.predict(X_train)

In [27]:
# for validation set
base_model_pred_valid = base_model.predict(X_valid)

In [28]:
base_model_pred.shape

(1881, 7, 7, 512)

In [30]:
base_model_pred_valid.shape

(471, 7, 7, 512)

In [31]:
# flatten model output to one dimension , training set
base_model_pred = base_model_pred.reshape(1881, 7*7*512)

In [33]:
base_model_pred.shape

(1881, 25088)

In [34]:
# flatten model output to one dimension for every sample of validation set
base_model_pred_valid = base_model_pred_valid.reshape(471,7*7*512)

In [35]:
base_model_pred_valid.shape

(471, 25088)

In [36]:
# check min max value from generated features
base_model_pred.min(), base_model_pred.max()

(0.0, 11.343674)

In [37]:
# normalize features generated from VGG16 model to (0,1)
max_value = base_model_pred.max()

base_model_pred = base_model_pred/max_value
base_model_pred_valid = base_model_pred_valid / max_value
base_model_pred.min() , base_model_pred.max()

(0.0, 1.0)

In [38]:
# create a sequential model
model = Sequential()
model.add(InputLayer((7*7*512,)))
model.add(Dense(units = 1024, activation = 'relu'))
model.add(Dense(units = 2 , activation = 'softmax'))

In [40]:
model.compile(optimizer = 'sgd', metrics = ['accuracy'], loss = 'categorical_crossentropy')

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [43]:
# train model using features generated from VGG16 model
model.fit(base_model_pred,y_train,epochs = 10 , validation_data = (base_model_pred_valid,y_valid))

Train on 1881 samples, validate on 471 samples
Epoch 1/10
1881/1881 [==============================] - 47s 25ms/step - loss: 0.6322 - accuracy: 0.6443 - val_loss: 0.5932 - val_accuracy: 0.6794
Epoch 2/10
1881/1881 [==============================] - 31s 16ms/step - loss: 0.5451 - accuracy: 0.7815 - val_loss: 0.5295 - val_accuracy: 0.7707
Epoch 3/10
1881/1881 [==============================] - 20s 11ms/step - loss: 0.4802 - accuracy: 0.8379 - val_loss: 0.4746 - val_accuracy: 0.8280
Epoch 4/10
1881/1881 [==============================] - 20s 10ms/step - loss: 0.4292 - accuracy: 0.8533 - val_loss: 0.4377 - val_accuracy: 0.8344
Epoch 5/10
1881/1881 [==============================] - 20s 11ms/step - loss: 0.3909 - accuracy: 0.8655 - val_loss: 0.4055 - val_accuracy: 0.8408
Epoch 6/10
1881/1881 [==============================] - 21s 11ms/step - loss: 0.3560 - accuracy: 0.8804 - val_loss: 0.3805 - val_accuracy: 0.8556
Epoch 7/10
1881/1881 [==============================] - 24s 13ms/step - loss:

## GET PREDICTIONS

In [44]:
pred = model.predict_classes(base_model_pred_valid)

In [45]:
pred

array([0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,